# !!! Make sure you run dcparking first !!!

In [43]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
import warnings
warnings.filterwarnings('ignore')

In [44]:
#load crime csv
csv_file = "Resources/2018 dc-crimes.csv"
dccrimes_df = pd.read_csv(csv_file)
dccrimes_df.head()

,NEIGHBORHOOD_CLUSTER,CENSUS_TRACT,offensegroup,LONGITUDE,END_DATE,offense-text,SHIFT,YBLOCK,DISTRICT,WARD,...,BLOCK,START_DATE,CCN,OFFENSE,OCTO_RECORD_ID,ANC,REPORT_DAT,METHOD,location,LATITUDE
0,cluster 20,9501.0,violent,-77.001338,2018-06-29T04:16:58.000,assault w/dangerous weapon,midnight,141428.0,4.0,5,...,290 - 399 block of taylor street ne,2018-06-29T01:33:41.000,18105959,assault w/dangerous weapon,18105959-01,5A,2018-06-29T07:24:04.000Z,gun,"38.940749726052005,-77.001340321018176",38.940742
1,cluster 6,10700.0,property,-77.041679,2018-06-29T03:27:13.000,theft f/auto,midnight,137809.0,2.0,2,...,1300 - 1399 block of 18th street nw,2018-06-29T02:46:03.000,18105973,theft f/auto,18105973-01,2B,2018-06-29T07:28:14.000Z,others,"38.908141188407896,-77.041681467729603",38.908133
2,cluster 8,5800.0,property,-77.022434,2018-06-29T02:58:41.000,burglary,day,137002.0,1.0,2,...,700 - 799 block of i street nw,2018-06-29T02:55:32.000,18106012,burglary,18106012-01,2C,2018-06-29T11:20:16.000Z,others,"38.900876728309974,-77.02243642733967",38.900869
3,cluster 1,3800.0,property,-77.040613,2018-06-16T18:27:19.000,theft/other,midnight,139054.0,3.0,1,...,2200 - 2260 block of champlain street nw,2018-06-16T17:36:17.000,18098009,theft/other,18098009-01,1C,2018-06-17T05:55:09.000Z,others,"38.919356945522274,-77.040615610052697",38.919349
4,cluster 8,5201.0,property,-77.031567,NaN,theft/other,evening,137555.0,3.0,2,...,10 - 10 block of thomas circle nw,2018-07-03T18:31:51.000,18108776,theft/other,18108776-01,2F,2018-07-03T23:19:56.000Z,others,"38.905856237775417,-77.031569074382645",38.905848


In [45]:
# list the columns
dccrimes_df.columns

Index(['NEIGHBORHOOD_CLUSTER', 'CENSUS_TRACT', 'offensegroup', 'LONGITUDE',
       'END_DATE', 'offense-text', 'SHIFT', 'YBLOCK', 'DISTRICT', 'WARD',
       'YEAR', 'offensekey', 'BID', 'sector', 'PSA', 'ucr-rank', 'BLOCK_GROUP',
       'VOTING_PRECINCT', 'XBLOCK', 'BLOCK', 'START_DATE', 'CCN', 'OFFENSE',
       'OCTO_RECORD_ID', 'ANC', 'REPORT_DAT', 'METHOD', 'location',
       'LATITUDE'],
      dtype='object')

In [46]:
#set the new columns we want to use.
newdccrimes_df = dccrimes_df[['LATITUDE','LONGITUDE','OFFENSE', 'REPORT_DAT']]
newdccrimes_df.head()

,LATITUDE,LONGITUDE,OFFENSE,REPORT_DAT
0,38.940742,-77.001338,assault w/dangerous weapon,2018-06-29T07:24:04.000Z
1,38.908133,-77.041679,theft f/auto,2018-06-29T07:28:14.000Z
2,38.900869,-77.022434,burglary,2018-06-29T11:20:16.000Z
3,38.919349,-77.040613,theft/other,2018-06-17T05:55:09.000Z
4,38.905848,-77.031567,theft/other,2018-07-03T23:19:56.000Z


## Getting date information

In [47]:
# clean up date to make it just month and day
dates = []
dates = newdccrimes_df['REPORT_DAT'].values
dates [1]

'2018-06-29T07:28:14.000Z'

In [48]:
# get the month

MM = []
for d in dates:
    MM.append(d[5:7])
print(MM)

['06', '06', '06', '06', '07', '06', '06', '06', '07', '07', '07', '07', '07', '05', '05', '05', '05', '05', '05', '05', '06', '07', '07', '05', '07', '05', '07', '05', '07', '05', '07', '05', '07', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '07', '07', '07', '07', '07', '07', '07', '07', '07', '07', '07', '07', '07', '07', '07', '07', '07', '07', '07', '07', '07', '07', '05', '05', '05', '05', '05', '05', '05', '05', '05', '05', '05', '05', '05', '05', '05', '05', '05', '05', '05', '05', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '07', '07', '07', '07', '07', '07', '07', '07', '07', '07', '07', '07', '07', '07', '07', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '06', '07', '07', '07', '07', '07', '07', '07', '07', '07', '07', '07', '07', '07

In [49]:
#get the day
DD = []
for d in dates:
    DD.append(d[8:10])
print(DD)

['29', '29', '29', '17', '03', '16', '17', '17', '09', '08', '13', '13', '13', '19', '19', '20', '19', '20', '20', '19', '12', '17', '16', '24', '16', '24', '20', '28', '20', '29', '21', '29', '24', '17', '05', '02', '17', '17', '02', '02', '21', '02', '02', '03', '07', '07', '08', '08', '08', '12', '12', '12', '12', '12', '12', '12', '22', '25', '25', '25', '26', '25', '26', '29', '29', '29', '29', '29', '04', '04', '04', '04', '04', '04', '08', '08', '08', '08', '08', '09', '08', '08', '09', '09', '13', '13', '13', '13', '14', '13', '19', '20', '20', '20', '20', '20', '20', '20', '20', '24', '24', '25', '24', '25', '25', '25', '25', '29', '29', '29', '02', '02', '03', '03', '03', '08', '08', '08', '13', '12', '13', '13', '18', '17', '17', '17', '20', '20', '20', '20', '21', '21', '24', '24', '24', '24', '24', '17', '17', '17', '22', '22', '22', '22', '26', '26', '26', '26', '26', '29', '29', '30', '30', '30', '04', '04', '04', '04', '09', '09', '18', '13', '13', '13', '14', '13', '14

In [50]:
#create the MONTH column and populate (you'll get an error but it works)
newdccrimes_df["MONTH"]=MM

In [51]:
#create the DAY column and populate (you'll get an error but it works)
newdccrimes_df["DAY"]=DD

In [75]:
#put date back together in YYYY-MM-DD format
newdccrimes_df['DATE']=[f"2018-{newdccrimes_df['MONTH'][i]}-{newdccrimes_df['DAY'][i]}" for i in newdccrimes_df.index]

In [53]:
#create new columns (getting rid of the report_dat)
finaldccrimes_df = newdccrimes_df[['LATITUDE','LONGITUDE','OFFENSE', 'MONTH', 'DAY', 'DATE']]
finaldccrimes_df.head()

,LATITUDE,LONGITUDE,OFFENSE,MONTH,DAY,DATE
0,38.940742,-77.001338,assault w/dangerous weapon,06,29,2018-06-29
1,38.908133,-77.041679,theft f/auto,06,29,2018-06-29
2,38.900869,-77.022434,burglary,06,29,2018-06-29
3,38.919349,-77.040613,theft/other,06,17,2018-06-17
4,38.905848,-77.031567,theft/other,07,03,2018-07-03


## Push to SQL

In [54]:

#Create Database
connection_string = "root:Password@localhost/DCDATA_DB?charset=utf8"
engine = create_engine(f'mysql://{connection_string}')

In [55]:
#load data after dropping columns from Crime
finaldccrimes_df.to_sql(name='CRIME', con=engine, if_exists='replace', index=False)

In [56]:
#confirm new crimes data loaded successfully
pd.read_sql_query('select * from CRIME', con=engine).head(5)

,LATITUDE,LONGITUDE,OFFENSE,MONTH,DAY,DATE
0,38.940742,-77.001338,assault w/dangerous weapon,06,29,2018-06-29
1,38.908133,-77.041679,theft f/auto,06,29,2018-06-29
2,38.900869,-77.022434,burglary,06,29,2018-06-29
3,38.919349,-77.040613,theft/other,06,17,2018-06-17
4,38.905848,-77.031567,theft/other,07,03,2018-07-03
5,38.873151,-76.984316,theft/other,06,16,2018-06-16
6,38.905024,-76.995538,theft/other,06,17,2018-06-17
7,38.895438,-77.016185,assault w/dangerous weapon,06,17,2018-06-17
8,38.912606,-77.024986,theft f/auto,07,09,2018-07-09
9,38.916991,-77.033244,theft/other,07,08,2018-07-08


## Start Comparing Crimes Against Parking Tickets

In [57]:
#Start groupby for crimes
groupybycrimes=finaldccrimes_df.groupby('DATE').count().reset_index()[["DATE", "OFFENSE"]]

In [58]:
#check data
groupybycrimes.head(5)

,DATE,OFFENSE
0,2018-01-01,36
1,2018-01-02,61
2,2018-01-03,80
3,2018-01-04,58
4,2018-01-05,71


In [64]:
#Pull in Parking data from SQL
newdcpark_df=pd.read_sql_query('select * from PARKING', con=engine)
newdcpark_df.head()

,Latitude,Longitude,VIOLATION_DESCRIPTION,TICKET_ISSUE_DATE,MONTH,DAY,DATE
0,38.88760034879,-76.993201829703,NO PARKING STREET CLEANING,2018-04-04T00:00:00.000Z,04,04,2018-04-04
1,38.8967425107048,-77.0189307155777,NO STOPPING OR STANDING IN PM RUSH HOUR ZONE,2018-04-04T16:08:00.000Z,04,04,2018-04-04
2,38.9072472840597,-77.0108629472506,NO PARKING STREET CLEANING,2018-04-04T10:04:00.000Z,04,04,2018-04-04
3,38.9126984450469,-77.07856940182,PARK WITHIN 20 FEET OF A BUS STOP OR ZONE,2018-04-04T16:31:00.000Z,04,04,2018-04-04
4,38.9232931808667,-77.0417595163018,FAIL TO DISPLAY A MULTISPACE METER RECEIPT,2018-04-04T19:22:00.000Z,04,04,2018-04-04


In [68]:
#Start Groupby for parking
groupbyparking=newdcpark_df.groupby('DATE').count().reset_index()[["DATE", "VIOLATION_DESCRIPTION"]]

In [69]:
#check data
groupbyparking.head(5)

,DATE,VIOLATION_DESCRIPTION
0,2018-01-01,124
1,2018-01-02,2284
2,2018-01-03,3764
3,2018-01-04,2009
4,2018-01-05,2420


In [70]:
#Merge the parking ticket DF and the crime data frame
mergedf = pd.merge(groupybycrimes,groupbyparking, how='inner'
                   ,left_on = 'DATE',right_on = 'DATE')

In [71]:
#Visualize DataFrame
mergedf.head()

,DATE,OFFENSE,VIOLATION_DESCRIPTION
0,2018-01-01,36,124
1,2018-01-02,61,2284
2,2018-01-03,80,3764
3,2018-01-04,58,2009
4,2018-01-05,71,2420


In [72]:
#Push Cleaned DF to SQL
mergedf.to_sql(name='Merged_Data', con=engine, if_exists='replace', index=False)

In [74]:
#Confirm it was pushed to SQL
pd.read_sql_query('select * from Merged_Data', con=engine).head()

,DATE,OFFENSE,VIOLATION_DESCRIPTION
0,2018-01-01,36,124
1,2018-01-02,61,2284
2,2018-01-03,80,3764
3,2018-01-04,58,2009
4,2018-01-05,71,2420
